# Random forest 

In [3]:
# python.exe -m pip install --upgrade pip
!python -m pip install statsmodels

## import des librairies l'environnement

In [7]:
#!pip3 uninstall -y Tensorflow 
# !pip3 install imblearn
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import preprocessing
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE

In [ ]:
# # Classe où vous voulez utiliser le DataFrame nettoyé
# class RandomForest:
#     def __init__(self):
#         # Importez la classe Nettoyage
#         from nettoyage import Nettoyage
# 
#         # Créez une instance de la classe Nettoyage
#         nettoyeur = Nettoyage()
# 
#         # Accédez au DataFrame nettoyé
#         df_nettoyé = nettoyeur.df_joint

## Data

In [2]:
data = pd.read_csv("/shared_data/notebook_jupyter/machine_learning/data/bank.txt", sep=',' )
data.columns

FileNotFoundError: [Errno 2] No such file or directory: '/shared_data/notebook_jupyter/machine_learning/data/bank.txt'

#####   y --> has the client subscribed a term deposit? (binary: “1” means “Yes”  , “0” means “No” )

In [3]:
data.head()

NameError: name 'data' is not defined

### The education column of the dataset has many categories and we need to reduce the categories for a better modelling

#### Let us group “basic.4y”, “basic.9y” and “basic.6y” together and call them “basic”

In [ ]:
print(data['education'].unique())
data['education']=np.where(data['education'] =='basic.9y', 'Basic', data['education'])
data['education']=np.where(data['education'] =='basic.6y', 'Basic', data['education'])
data['education']=np.where(data['education'] =='basic.4y', 'Basic', data['education'])
print(data['education'].unique())

## Create dummy variables

In [ ]:
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1
    
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

In [ ]:
## Our final data columns will be:
data_final=data[to_keep]
data_final.columns.values

In [ ]:
data_final.shape

## Over-sampling using SMOTE

In [ ]:
X = data_final.loc[:, data_final.columns != 'y']
y = data_final.loc[:, data_final.columns == 'y']

from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])

# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['y']==0]))
print("Number of subscription",len(os_data_y[os_data_y['y']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

## Recursive Feature Elimination

In [ ]:
data_final_vars=data_final.columns.values.tolist()
y=['y']
X=[i for i in data_final_vars if i not in y]

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
features  = pd.DataFrame(list(zip(data_final_vars , rfe.support_, rfe.ranking_)), columns =['var','support', 'ranking'])
features_model= features.loc  [  (features['support']== True)   ,  :] 
# features_model

In [ ]:
cols = features_model['var'].tolist()
print(cols)
X=os_data_X[cols]
y=os_data_y['y']
# y =pd.DataFrame(os_data_y['y'])

In [ ]:
# cols=['job_unknown', 'marital_divorced', 'marital_married', 'marital_single','marital_unknown', 'education_Basic'
#       , 'education_illiterate' , 'education_professional.course' , 'education_university.degree' , 'housing_no' , 
#       'housing_unknown'  , 'housing_yes' , 'loan_no' , 'month_sep' , 'day_of_week_fri' , 'day_of_week_mon'  , 
#       'day_of_week_thu'  , 'day_of_week_tue']
# ## 'default_yes' , 'loan_unknown'
     
# X=os_data_X[cols]
# y=os_data_y['y']

## Implementing the model

In [ ]:
########################### Random forest 

# Import the random forest model
from sklearn.ensemble import RandomForestClassifier 
## This line instantiates the model
rf = RandomForestClassifier() 
## Fit the model 
rf.fit(X, y) 


## Importance RF
feature_importances = pd.DataFrame(rf.feature_importances_ ,index = X.columns, columns=['importance']).sort_values('importance',  ascending=False)
display(feature_importances.iloc[0:20,:])

importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
indices = np.argsort(importances)[::-1]


## presentation graphique de 'Importance RF'
feat_importances = pd.Series(rf.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')




## Random Forest Model Fitting

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=0)
rf = RandomForestClassifier() 
rf.fit(X_train, y_train)

### Predicting the test set results and calculating the accuracy

In [ ]:
y_pred = rf.predict(X_test)
print('Accuracy of Random Forest classifier on test set: {:.2f}'.format(rf.score(X_test, y_test)))

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
# confusion_matrix = confusion_matrix(y_test, y_pred)
# print(confusion_matrix)
df_confusion = pd.crosstab(y_test, y_pred)
print(df_confusion)

## ROC Curve

In [ ]:
### model regression logistique 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)


from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

logit_roc_auc1 = roc_auc_score(y_test, rf.predict(X_test))
logit_roc_auc2 = roc_auc_score(y_test, logreg.predict(X_test))

fpr1, tpr1, thresholds1 = roc_curve(y_test, rf.predict_proba(X_test)[:,1])
fpr2, tpr2, thresholds2 = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])

plt.figure()
plt.plot(fpr1, tpr1, label='Random Forest (area = %0.2f)' % logit_roc_auc1)
plt.plot(fpr2, tpr2, label='Logistic Regression (area = %0.2f)' % logit_roc_auc2)

plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

